In [3]:
import numpy as np 


In [5]:
import yfinance as yf 

In [9]:
#install web scraping capabilities 
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options 
from webdriver_manager.chrome import ChromeDriverManager
import time 
import random

In [11]:
import pandas as pd 

In [13]:
chrome_options = Options()
service = Service(ChromeDriverManager().install())
browser = webdriver.Chrome()
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

In [15]:
browser.get(url)
#find S&P 500 table
table = browser.find_element(By.XPATH, '//*[@id="constituents"]')

#get all rows in the table
rows = table.find_elements(By.TAG_NAME, "tr")

#create ticker list
tickers = []

 
for row in rows[1:]:
    cols = row.find_elements(By.TAG_NAME, "td")
    if cols:
        ticker = cols[0].text.strip().replace('.', '-')  # Clean for Yahoo Finance format
        tickers.append(ticker)


browser.quit()

#display the ticker list 
print(tickers[:10])  

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']


In [17]:
count = len(tickers)
print(count)

503


# Now that we have the list we can loop through that list when gathering data from yfinance 

In [102]:
financial_data = []

for i, ticker in enumerate(tickers):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        data = {
            'Ticker': ticker,
            'Name': info.get('shortName'),
            'P/E Ratio': info.get('trailingPE'),
            'Forward P/E': info.get('forwardPE'),
            'Revenue (TTM)': info.get('totalRevenue'),
            'Operating Income': info.get('operatingIncome'),
            'Gross Profits': info.get('grossProfits'),
            'Net Income': info.get('netIncomeToCommon'),
            'EBITDA': info.get('ebitda'),
            'Market Cap': info.get('marketCap'),
            'Sector': info.get('sector'),
        }

        financial_data.append(data)
        time.sleep(0.5)  # polite delay to avoid rate limiting

    except Exception as e:
        print(f"Error for {ticker}: {e}")

# Convert to DataFrame
sp_data = pd.DataFrame(financial_data)
print(sp_data.head())


  Ticker                    Name  P/E Ratio  Forward P/E  Revenue (TTM)  \
0    MMM              3M Company  17.295145    17.579748    24512999424   
1    AOS  A.O. Smith Corporation  17.988981    16.044226     3818099968   
2    ABT     Abbott Laboratories  16.912418    25.073645    42344001536   
3   ABBV             AbbVie Inc.  75.154160    14.869744    56334000128   
4    ACN           Accenture plc  24.043730    20.711443    67221491712   

  Operating Income  Gross Profits   Net Income        EBITDA    Market Cap  \
0             None    10101999616   4419999744  5.189000e+09   74742579200   
1             None     1456099968    533600000  7.775000e+08    9372443648   
2             None    23677999104  13450999808  1.100250e+10  224386334720   
3             None    39644999680   4238000128  2.671800e+10  319070896128   
4             None    21633724416   7683452928  1.147250e+10  182430236672   

        Sector  
0  Industrials  
1  Industrials  
2   Healthcare  
3   Healthca

In [104]:
sp_data.to_csv('SP_Financial_Data.csv')

In [116]:
#cleaning the data
sp_data_new =sp_data.drop('Operating Income', axis=1)
sp_data_new
sp_data_new.isna().sum()
sp_data_new = sp_data_new.dropna()
sp_data_new.rename(columns={'Name': 'Company'}, inplace = True)
sp_data_new
                   

,Ticker,Company,P/E Ratio,Forward P/E,Revenue (TTM),Gross Profits,Net Income,EBITDA,Market Cap,Sector
0,MMM,3M Company,17.295145,17.579748,24512999424,10101999616,4419999744,5.189000e+09,74742579200,Industrials
1,AOS,A.O. Smith Corporation,17.988981,16.044226,3818099968,1456099968,533600000,7.775000e+08,9372443648,Industrials
2,ABT,Abbott Laboratories,16.912418,25.073645,42344001536,23677999104,13450999808,1.100250e+10,224386334720,Healthcare
3,ABBV,AbbVie Inc.,75.154160,14.869744,56334000128,39644999680,4238000128,2.671800e+10,319070896128,Healthcare
4,ACN,Accenture plc,24.043730,20.711443,67221491712,21633724416,7683452928,1.147250e+10,182430236672,Technology
...,...,...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,31.673971,24.338947,8561999872,3212000000,890000000,1.683000e+09,28133576704,Industrials
499,YUM,"Yum! Brands, Inc.",28.272032,24.273027,7549000192,3584000000,1486000000,2.714000e+09,41103540224,Consumer Cyclical
500,ZBRA,Zebra Technologies Corporation,23.976423,15.216957,4981000192,2412999936,528000000,9.370000e+08,12483056640,Technology
501,ZBH,"Zimmer Biomet Holdings, Inc.",22.911964,11.802325,7678600192,5498899968,903800000,2.590300e+09,20080861184,Healthcare


In [128]:
# read in the esg_data 
esg_data = pd.read_csv('esg_data_full.csv')
esg_data.rename(columns={'Ticker_2': 'Ticker'}, inplace=True)
esg_data_new = esg_data.drop('Unnamed: 0', axis = 1)
esg_data_new = esg_data.drop('Ticker_1', axis = 1)
esg_data_new


,Unnamed: 0,Company,Ticker,ESG score,ESG risk
0,0,"1-800-FLOWERS.COM, Inc.",FLWS,28.9,Medium
1,1,1&1 AG,1U1,27.7,Medium
2,2,"10X Genomics, Inc.",TXG,22.5,Medium
3,3,11 Bit Studios SA,11B,16.3,Low
4,4,1st Source Corp.,SRCE,35.0,High
...,...,...,...,...,...
13979,13979,Zydus Wellness Ltd.,531335,27.9,Medium
13980,13980,"Zylox-Tonbridge Medical Technology Co., Ltd.",2190,25.6,Medium
13981,13981,"Zymeworks, Inc.",ZYME,29.0,Medium
13982,13982,"Zynex, Inc.",ZYXI,32.0,High


In [130]:
esg_data_new = esg_data_new.drop('Unnamed: 0' , axis = 1) 
esg_data_new

,Company,Ticker,ESG score,ESG risk
0,"1-800-FLOWERS.COM, Inc.",FLWS,28.9,Medium
1,1&1 AG,1U1,27.7,Medium
2,"10X Genomics, Inc.",TXG,22.5,Medium
3,11 Bit Studios SA,11B,16.3,Low
4,1st Source Corp.,SRCE,35.0,High
...,...,...,...,...
13979,Zydus Wellness Ltd.,531335,27.9,Medium
13980,"Zylox-Tonbridge Medical Technology Co., Ltd.",2190,25.6,Medium
13981,"Zymeworks, Inc.",ZYME,29.0,Medium
13982,"Zynex, Inc.",ZYXI,32.0,High


In [174]:
# Step 1: Merge on ticker only
merged_df = pd.merge(sp_data_new, esg_data_new, on='Ticker', how='left', suffixes=('_sp', '_esg'))

# Step 2: Use fuzzy matching to keep rows where names are similar
from difflib import SequenceMatcher

def is_similar(name1, name2, threshold=0.45):
    return SequenceMatcher(None, str(name1), str(name2)).ratio() > threshold

# Step 3: Apply the fuzzy match filter
merged_df['Name_Match'] = merged_df.apply(lambda row: is_similar(row['Company_sp'], row['Company_esg']), axis=1)

# Step 4: Keep only the ones with a strong match
Final_merged_df = merged_df[merged_df['Name_Match']]
Final_merged_df = Final_merged_df.drop('Name_Match', axis = 1)
Final_merged_df.to_csv('Final_Merged.csv')



In [176]:
Final_merged_df.to_excel("Merged.xlsx", index=False)


In [180]:
ticker_counts = Final_merged_df['Ticker'].value_counts()
duplicates = ticker_counts[ticker_counts > 1]
print(duplicates)
# I fixed these in excel no clue how I would fix on python 

Ticker
SPG     3
CF      2
HAS     2
DOW     2
FAST    2
CMG     2
HUM     2
MRK     2
PRU     2
RCL     2
APA     2
Name: count, dtype: int64


In [182]:
model_df = pd.read_csv('Model.csv')
model_df
                       

,Ticker,Company,P/E Ratio,Forward P/E,Revenue (TTM),Gross Profits,Net Income,EBITDA,Market Cap,Sector,ESG score,ESG risk
0,MMM,3M Company,17.295145,17.579748,2.451300e+10,1.010200e+10,4.420000e+09,5.189000e+09,7.474258e+10,Industrials,43.0,Severe
1,AOS,A.O. Smith Corporation,17.988981,16.044226,3.818100e+09,1.456100e+09,5.336000e+08,7.775000e+08,9.372444e+09,Industrials,26.8,Medium
2,ABT,Abbott Laboratories,16.912418,25.073645,4.234400e+10,2.367800e+10,1.345100e+10,1.100250e+10,2.243860e+11,Healthcare,20.4,Medium
3,ABBV,AbbVie Inc.,75.154160,14.869744,5.633400e+10,3.964500e+10,4.238000e+09,2.671800e+10,3.190710e+11,Healthcare,24.3,Medium
4,ACN,Accenture plc,24.043730,20.711443,6.722149e+10,2.163372e+10,7.683453e+09,1.147250e+10,1.824300e+11,Technology,11.1,Low
...,...,...,...,...,...,...,...,...,...,...,...,...
438,XYL,Xylem Inc.,31.673971,24.338947,8.562000e+09,3.212000e+09,8.900000e+08,1.683000e+09,2.813358e+10,Industrials,22.9,Medium
439,YUM,"Yum! Brands, Inc.",28.272032,24.273027,7.549000e+09,3.584000e+09,1.486000e+09,2.714000e+09,4.110354e+10,Consumer Cyclical,20.5,Medium
440,ZBRA,Zebra Technologies Corporation,23.976423,15.216957,4.981000e+09,2.413000e+09,5.280000e+08,9.370000e+08,1.248306e+10,Technology,9.9,Negligible
441,ZBH,"Zimmer Biomet Holdings, Inc.",22.911964,11.802325,7.678600e+09,5.498900e+09,9.038000e+08,2.590300e+09,2.008086e+10,Healthcare,26.2,Medium


In [184]:
model_df.isna().sum()


Ticker           0
Company          0
P/E Ratio        0
Forward P/E      0
Revenue (TTM)    0
Gross Profits    0
Net Income       0
EBITDA           0
Market Cap       0
Sector           0
ESG score        0
ESG risk         0
dtype: int64

In [186]:
model_df.duplicated().sum()

np.int64(0)